In [11]:
import sys
import os
import json
import google.generativeai as genai
import os
import time
import ipynb_path
from surrealdb import Surreal;

from IPython.display import display, Markdown


#get this notebook's path for access to the other files needed
dir_path = os.path.dirname(os.path.realpath(ipynb_path.get(__name__)))
sys.path.append(dir_path) #add the current directory for adding py imports
from RAG_prompts import STD_RAG_PROMT, GRAPH_RAG_PROMT

In [3]:

# this folder
nb_folder = dir_path
input_file = nb_folder + "/Operation Dulce v2 1 1.md"

    
ip = "0.0.0.0:8080"
url = "ws://{0}/rpc".format(ip)

u = "root"
p = "root"
n = "graph_rag"
d = "graph_rag"
db_folder = nb_folder + "/db"

surrealdb_start = "surreal start --log none --user {u} --pass {p} --bind {ip} \"rocksdb://{db_folder}\"".format(
    u=u,
    p=p,
    ip=ip,
    db_folder=db_folder)

#run this command if your surreal instance isn't running yet 
#copy and paste from below into a terminal
print(surrealdb_start)


surreal start --log none --user root --pass root --bind 0.0.0.0:8080 "rocksdb:///Users/sandro/Library/CloudStorage/GoogleDrive-apireno@gmail.com/My Drive/code/graph_rag/db"


In [6]:

# get the text from the source file for use in our prompt
with open(input_file, "r") as f:
    input_text = f.read()


retrieve_function_surql = """
DEFINE FUNCTION IF NOT EXISTS fn::get_related_entities($text: string) {
    
    LET $emb = fn::get_embeddings($text);
    
    RETURN SELECT id,description, ->RELATED_TO.{out,strength,description} FROM PERSON 
        WHERE  embedding <|5,COSINE|> $emb;
};
"""


async with Surreal(url) as db:
    await db.signin({"user": u, "pass": p}) 
    await db.use(n, d)
    outcome = await db.query(retrieve_function_surql)



In [8]:
#this function will call the LLM with a prompt that includes the text of the original document
def get_std_LLM_response(the_model,the_question,the_input_text):

    messages = [
            {"role": "user", "parts": [{"text": STD_RAG_PROMT.format(
                question = the_question,
                documents = the_input_text
            )}]}
        ]

    
    response = model.generate_content(messages)
    return response
    
#this function will call the LLM with a prompt that includes the text of the original document
#plus the relevant entities and relationships from the knowledge graph bases on a vector search
async def get_graph_RAG_response(the_db,the_model,the_question,the_input_text):
    
    get_the_graph_context_surql = "fn::get_related_entities('{0}')"
    
    outcome = await the_db.query(get_the_graph_context_surql.format(the_question))

    messages = [
            {"role": "user", "parts": [{"text": GRAPH_RAG_PROMT.format(
                question = the_question,
                documents = the_input_text,
                knowledge_graph = outcome
            )}]}
        ]

    
    response = model.generate_content(messages)
    return response
    
       

In [9]:

# Set your API key (I've stored mine in an env variable)
genai.configure(api_key=os.getenv('GOOGLE_GENAI_API_KEY'))

#flash is faster and good enough
#model = "gemini-1.0-pro"
model = "gemini-1.5-flash"

model = genai.GenerativeModel(model)



In [12]:
#Lets test a question...

q = "Who is the main character and what motivates them?"

response = get_std_RAG_response(model,q,input_text)

print ("\n\n\n\n ------ STD RAG -------------------------\n\n\n\n")



display(Markdown(response.text))

print ("\n\n\n\n ------ GRAPH RAG -------------------------\n\n\n\n")

async with Surreal(url) as db:
    await db.signin({"user": u, "pass": p}) 
    await db.use(n, d)
    response = await get_graph_RAG_response(db,model,q,input_text)
    display(Markdown(response.text))








 ------ STD RAG -------------------------






While the story doesn't explicitly focus on one singular protagonist, **Agent Alex Mercer** emerges as a central figure. He drives the narrative with his:

* **Military discipline:** He  is a veteran of Paranormal Military Squad, adhering to protocol while also questioning the limitations of established procedures.
* **Strategic mind:** He analyzes the situation, strategizes for engagement, and prioritizes understanding over confrontation.
* **Humanity:** He acknowledges the importance of cautious optimism and a willingness to learn from the alien intelligence. 

His motivation seems to stem from a combination of:

* **Duty:**  He's dedicated to his mission and understands the significance of the team's role. 
* **Curiosity:** He's driven by a desire to understand the unknown and the possibility of forging a new path for humanity.
* **Responsibility:** He recognizes the potential ramifications of their actions and wants to ensure that humanity makes a good first impression on the alien intelligence. 






 ------ GRAPH RAG -------------------------






The main character in this story is **Agent Alex Mercer**. He is a member of the Paranormal Military Squad, and he is motivated by a deep-seated desire to understand the unknown. He is analytical and determined, and he often acts as a leader for the team, prioritizing caution and diplomacy while pushing for deeper understanding. He is also driven by a sense of responsibility to protect humanity from any potential threats posed by the alien intelligence. 


In [13]:

#Let's test a question that asks about the main character's relationships
q = "What places and people does Alex Mercer hold dearest?"

response = get_std_RAG_response(model,q,input_text)

print ("\n\n\n\n ------ STD RAG -------------------------\n\n\n\n")


display(Markdown(response.text))

print ("\n\n\n\n ------ GRAPH RAG -------------------------\n\n\n\n")

async with Surreal(url) as db:
    await db.signin({"user": u, "pass": p}) 
    await db.use(n, d)
    response = await get_graph_RAG_response(db,model,q,input_text)
    display(Markdown(response.text))







 ------ STD RAG -------------------------






The documents provided don't directly address Alex Mercer's personal life, including the places and people he holds dearest. While they describe his character and his actions during Operation: Dulce, they mainly focus on his professional persona as an agent and his interactions with his team members. 






 ------ GRAPH RAG -------------------------






Based on the provided documents and knowledge graph, we can infer that Alex Mercer holds the following places and people dearest:

**Places:**

* **Earth:** Alex Mercer is dedicated to protecting Earth from any potential threats arising from the alien contact.
* **Humanity:** He is deeply committed to protecting Humanity from any potential threats during the alien contact.
* **Dulce Base:**  While he is cautious and apprehensive, he is determined to unravel the secrets of Dulce Base.

**People:**

* **Jordan Hayes:** Alex values Jordan's analytical mind and their ability to see patterns, seeing them as crucial to their survival. He acknowledges Jordan's expertise and often seeks their input.
* **Sam Rivera:** Alex mentors and encourages Sam, recognizing their potential for deciphering complex signals and technology. He trusts Sam's intuition and analytical abilities.

It is important to note that the documents primarily focus on Alex's professional life, and there is no mention of his personal life or any romantic relationships. Therefore, we cannot definitively identify any specific people who he holds "dearest" in a personal sense. 


In [15]:



q = "Provide a timeline for Alex Mercers actions"

response = get_std_RAG_response(model,q,input_text)

print ("\n\n\n\n ------ STD RAG -------------------------\n\n\n\n")

display(Markdown(response.text))

print ("\n\n\n\n ------ GRAPH RAG -------------------------\n\n\n\n")

async with Surreal(url) as db:
    await db.signin({"user": u, "pass": p}) 
    await db.use(n, d)
    response = await get_graph_RAG_response(db,model,q,input_text)
    display(Markdown(response.text))






 ------ STD RAG -------------------------






## Alex Mercer's Timeline: 

**Chapter 1:**
- Attends initial briefing, seemingly compliant with protocols.
- Shows a quiet determination beneath the surface, sensing the unknown.
- Recognizes Jordan's analytical skills as crucial for survival.

**Chapter 2:**
- Expresses concern about the lack of communication from Dulce.
- Emphasizes the need for intuition and trust beyond technology.
- Acknowledges Sam's intuitive understanding of the data.

**Chapter 3:**
- Discusses the larger implications of the mission, beyond its immediate parameters.
- Notes the competitive tension with Taylor Cruz.
- Acknowledges Taylor's shift towards respect for the gravity of their work.

**Chapter 4:**
- Feels pulled into an unknown game, potentially as a pawn.
- Acknowledges the eerie atmosphere of Dulce.
- Confronts Taylor about the need to consider all possibilities. 

**Chapter 5:**
- Recognizes Jordan's understanding of the star alignments and its implications.
- Debates Taylor about the importance of hypothesis over blind obedience.
-  Acknowledges Sam's potential as a sentinel in the face of the unknown.

**Chapter 6:**
- Understands the shift in their mission from simple recovery to guarding a cosmic warning.
- Acknowledges the transformation in Jordan from doubt to action.
- Feels a sense of brotherhood with Jordan and Sam.

**Chapter 7:**
- Expresses concern about the evolving signal's potential for danger.
-  Reassures the team to proceed with cautious optimism.
- Acknowledges the changing nature of the mission from observation to interaction.

**Chapter 8:**
-  Emphasizes the need for caution and understanding when encountering the alien intelligence.
-  Recognizes the shift in their roles from operatives to guardians of a cosmic message.

**Chapter 9:**
-  Emphasizes the need for vigilance and awareness of the signal's intentions.
-  Acknowledges the potential for both breakthrough and disaster.
-  Reassures Jordan of his trust and support.

**Chapter 10:**
-  Urges the team to understand the intent behind the cosmic warning.
-  Recognizes the transformation of their mission from exploration to engagement.
-  Acknowledges the shift in their role from observers to active participants.

**Chapter 11:**
-  Urges caution in interacting with the potentially intelligent signal.
-  Emphasizes the importance of understanding over confrontation.
-  Acknowledges the need for protocols when establishing dialogue with an alien intelligence.

**Chapter 12:**
-  Recognizes the potential for diplomacy with the alien intelligence.
-  Acknowledges Taylor's caution about potential threats.
-  Emphasizes the importance of remaining grounded while seeking answers.

**Chapter 13:**
-  Urges the team to remain adaptable in the face of the unknown.
-  Understands and acknowledges the potential for cosmic negotiations.
-  Reminds the team that they are not just observers but participants in a larger dialogue.

**Chapter 14:**
-  Emphasizes the need for finesse in communicating with the alien intelligence.
-  Recognizes the transformation of their mission from simple recovery to a cosmic concerto.
-  Acknowledges the importance of collective expertise and teamwork.

**Chapter 15:**
-  Emphasizes the need for consistency and a clear message when communicating.
-  Recognizes the potential for both forging a bond and exposing vulnerabilities.
-  Urges the team to harmonize their response with the alien intelligence.

**Chapter 16:**
-  Reminds the team of the importance of prudence and caution.
-  Acknowledges the potential for both understanding and danger.
-  Urges the team to engage with the alien intelligence with care and respect.

This timeline shows Alex Mercer's progression from a cautious, protocol-driven agent to a leader who embraces the unknown and fosters a sense of collaborative discovery and diplomacy.  His actions demonstrate a growing understanding of the cosmic implications of their mission and a commitment to protecting humanity while seeking knowledge. 






 ------ GRAPH RAG -------------------------






## Timeline of Alex Mercer's Actions:

**Chapter 1:**

* **Initial Meeting:** Alex Mercer is present in the briefing room for Operation: Dulce. He is initially reluctant to engage, but eventually complies with Agent Taylor Cruz's directive to follow protocol.
* **Hidden Determination:** Despite his apparent compliance, Alex maintains a determined undercurrent, showing he won't be easily swayed by the challenges ahead.

**Chapter 2:**

* **Intuition and Trust:**  Alex visits Sam Rivera in their office, emphasizing the importance of intuition and trust in addition to technology. He recognizes Sam's talent for deciphering alien signals and instills a sense of trust in them.

**Chapter 3:**

* **Humanity's Stake:** Alex engages in conversation with Dr. Jordan Hayes, discussing the broader implications of their mission and the potential impact on humanity. They share a sense of the enormity of the task at hand.
* **Cautious Acceptance:** Alex observes Taylor Cruz’s initial dismissal of their conversation, but later notes a slight shift in Taylor's attitude, signaling a reluctant acceptance of the importance of their work.

**Chapter 4:**

* **Sense of Being Played:**  Alex shares with Jordan Hayes his feeling that they are being manipulated by unknown forces within the Dulce Base.
* **Unwavering Resolve:** Alex, facing Taylor Cruz’s authority, asserts his determination to proceed with the mission.

**Chapter 5:**

* **Cosmic Appointment:** Alex reacts to Jordan Hayes' theory about celestial alignments impacting the mission,  showing a willingness to consider the extraordinary. 
* **Openness to Unknowns:** Alex defends Jordan's theory against Taylor Cruz’s dismissal, demonstrating a willingness to embrace the unknown.

**Chapter 6:**

* **The Warning:**  Alex recognizes the importance of understanding the alien warning decoded by Jordan and Sam.
* **Shift in Mandate:** Alex acknowledges the change in their mission from mere observation to active participation.

**Chapter 7:**

* **First Contact:** Alex witnesses the team's initial contact with the alien intelligence and the profound change it brings. 
* **Embrace of Change:** Alex recognizes the need for a new approach—one of connection and comprehension—in dealing with the alien intelligence.

**Chapter 8:**

* **Navigating the Unknown:** Alex discusses with Jordan Hayes the importance of being prepared for the unknown, demonstrating his strategic thinking.
* **Collective Responsibility:** Alex acknowledges that their actions now affect humanity's place in the cosmos.

**Chapter 9:**

* **Aware and Reactive:** Alex probes Dr. Jordan Hayes about the alien intelligence's level of awareness and its potential intentions.
* **Cautious Optimism:** Alex encourages Jordan to continue communicating but warns against complacency in their communication.

**Chapter 10:**

* **Learning the Song:** Alex acknowledges the importance of understanding the alien signal's rhythmic nature and encourages his team to learn its language.
* **Humanity’s Reply:** Alex initiates a response to the alien signal, demonstrating his leadership and belief in the mission.

**Chapter 11:**

* **The First Emissaries:** Alex recognizes the monumental nature of their actions and their role as Earth's first emissaries to an alien civilization.

**Chapter 12:**

* **Empathy as a Bridge:** Alex suggests using empathy as a tool to bridge the gap between humanity and the alien intelligence.
* **Measured Response:** Alex stresses the need for a cautious and measured response to the alien communication.
* **Initiation of Dialogue:** Alex gives the order to initiate a formal communication with the alien intelligence.

**Chapter 13:**

* **Navigating the Unknown:** Alex acknowledges that their team is now venturing into uncharted territory and the need to adapt their approach.

**Chapter 14:**

* **Symphonic Dialogue:** Alex encourages his team to see their communication with the alien intelligence as a dance, a collaborative effort.
* **Earth’s First Note:** Alex recognizes the historical significance of their mission and the impact it will have on the future of humanity.

**Chapter 15:**

* **Seeking Harmony:** Alex directs his team to harmonize their responses with the alien signal, emphasizing the need for a collaborative approach.
* **Humanity’s Voice:** Alex acknowledges the importance of Earth's voice in the galactic conversation and encourages his team to make it heard. 

**Overall:** Throughout the chapters, Alex Mercer emerges as a strong leader, demonstrating both caution and resolve. He is driven by a sense of responsibility and a belief in humanity's potential to engage with the unknown. He fosters collaboration and respects the expertise of his team while ensuring the mission's overall success. His actions highlight his commitment to understanding and engaging with the alien intelligence, while safeguarding the interests of humanity. 
